## 제한조건:
- 글이 방금 올라와 감정표현이 3개가 되지 않은 게시물이 있으면
- 오류가 난다.
- default값인 Huffpost 같은 경우 보통 5~10분 정도가 지나면 감정표현 3개가 다 찬다.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
def page_crawling(id, pw, page_url="https://www.facebook.com/huffpost", date="2018114"):
    '''
    id: 페이스북 아이디
    pw: 페이스북 패스워드
    page_url: 크롤링하고 싶은 페이지 URL
    date: 언제 올라온 게시물 부터 크롤링 할 것인지
    '''
    
    # chorme 옵션, facebook을 들어갈 때 페이스북에서 알람을 보내는 것을 허용하겠냐는 chrome 알람이 뜨는데 
    # 이 알람은 alter로 처리할 수 있는 알람과는 다른 종류의 알람이므로 chromeoptions를 통해 알람이
    # 뜨지 않도록 한다.
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(chrome_options=chrome_options)
    
    # facebook 접속
    driver.get("https://www.facebook.com")
    time.sleep(3)
    
    # id와 pw를 입력하고 로그인 버튼을 누른다.
    driver.find_element_by_css_selector("#email").send_keys(id)
    driver.find_element_by_css_selector("#pass").send_keys(pw)
    driver.find_element_by_css_selector("#loginbutton").click()
    time.sleep(3)
    
    # 크롤링하고 싶은 페이지로 이동한다.
    driver.get(page_url)
    time.sleep(3)
    
    # 페이지 관리자와 채팅 할 수 있는 조그만 창이 밑에 뜨는데 그것을 꺼지게 하는 코드
    while True:
        try:
            driver.find_element_by_css_selector("._32ca").click()
            break
        except:
            time.sleep(3)
            
    # 페이지 내의 게시물 탭으로 이동
    driver.find_element_by_css_selector("#u_0_x > div:nth-child(5) > a > span._2yav").click()
    
    first_date = int(date)
    dates_list = []
    contents_list = []
    comments_list = []
    emotions_list = []
    df = pd.DataFrame(columns=["날짜", "내용", "댓글", "공유", "좋아요", "화나요", "웃겨요", "멋져요", "슬퍼요", "최고예요"])
    
    # 페이스북은 밑으로 내려야만 그 다음 게시물이 뜨기 때문에 초기에 입력한 날짜가 끝날 때까지 밑으로 내려가는 코드
    while True:
        try: 
            driver.find_element_by_css_selector("#www_pages_reaction_see_more_unitwww_pages_posts").click()
            dates = driver.find_elements_by_css_selector("#pagelet_timeline_main_column ._4-u2._4-u8 ._4-u2._4-u8 ._5pcq abbr")
            date = dates[-1].get_attribute("title")[:9].replace("-","")
            if int(date) < int(first_date):
                break
        except:
            time.sleep(3)
            
    # 게시물이 올라온 날짜를 크롤링 해서 리스트에 넣는 코드        
    dates = driver.find_elements_by_css_selector("#pagelet_timeline_main_column ._4-u2._4-u8 ._4-u2._4-u8 ._5pcq abbr")
    for date in dates:
        if int(date.get_attribute("title")[:9].replace("-","")) >= first_date:
            dates_list.append(date.get_attribute('title'))
    nums = len(dates_list)
    
    # 게시물에 존재하는 Text
    contents = driver.find_elements_by_css_selector("#pagelet_timeline_main_column ._4-u2._4-u8 ._4-u2._4-u8 .userContent p")
    for content in contents[:nums]:
        contents_list.append(content.text)
        
    # 댓글과 공유 개수    
    comments = driver.find_elements_by_css_selector("#pagelet_timeline_main_column ._4-u2._4-u8 ._4-u2._4-u8 ._ipn.clearfix._-5d ._ipo a")
    for comment in comments[:nums*2]:
        comments_list.append(comment.text)
    
    # 상위 3개의 감정표현과 그 개수
    emotions = driver.find_elements_by_css_selector("#pagelet_timeline_main_column ._4-u2._4-u8 ._4-u2._4-u8 ._ipn.clearfix._-5d ._ipp a")
    for emotion in emotions[:nums*4]:
        emotions_list.append(emotion.get_attribute("aria-label"))
        
    # 크롤링 한 list를 변환하여 데이터 프레임에 추가하는 코드
    for i in range(0, nums):
        date = dates_list[i]
        content = contents_list[i]
        comment = comments_list[2*i:2*i+2]
        comment[0] = comment[0][:-1]
        comment[1] = comment[1][:-1]
        emotion = emotions_list[4*i:4*i+4]
        emotion[0] = emotion[0][:-1]
        emotion[1] = emotion[1][:-1]
        emotion[2] = emotion[2][:-1]
            
        dic = {
            "날짜": date,
            "내용": content,
            "댓글": comment[0][3:],
            "공유": comment[1][3:],
            emotion[0].split(" ")[0]: emotion[0].split(" ")[1] ,
            emotion[1].split(" ")[0]: emotion[1].split(" ")[1],
            emotion[2].split(" ")[0]: emotion[2].split(" ")[1],
        }
        df = df.append(dic, ignore_index=True)
    
    # NULL값 처리
    df = df.fillna(0)
    df = df.astype(str)

    # 개수가 천이 넘어가면 1천, 2.1천 이런 식으로 보여지는데 천자를 지우고 1000을 곱해주는 코드
    for column in list(df.columns):
        for i in range(nums):
            if df[column][i][-1] == "천":
                df[column][i] = float(df[column][i][:-1]) * 1000
            elif df[column][i][-1] == "만":
                df[column][i] = float(df[column][i][:-1]) * 10000
    
    df.공유 = df.공유.apply(lambda x: x.replace(",",""))
    
    return df

In [3]:
df = page_crawling(ID, PW)

In [4]:
df

,날짜,내용,댓글,공유,좋아요,화나요,웃겨요,멋져요,슬퍼요,최고예요
0,2018-11-5 오전 01:02,"""Change, like success, is not a destination, i...",3,2,43,0,1,0,0,3
1,2018-11-5 오전 01:00,"The singer released her new song ""Thank U, Nex...",11,2,41,0,6,0,0,2
2,2018-11-5 오전 12:45,"Kanye West is quitting politics, but will he s...",54,5,23,8,24,0,0,0
3,2018-11-5 오전 12:30,The horrifying scramble of migrants looks surp...,43,177,537,0,401,0,0,47
4,2018-11-5 오전 12:00,"At Panera Bread , we believe an egg should be ...",15,21,61,0,0,6,0,5
5,2018-11-4 오후 11:44,He used the phrase while campaigning for Ron D...,340,589,127,1300,0,359,0,0
6,2018-11-4 오후 11:15,Princess Ayako has surrendered her royal statu...,63,63,425,0,0,41,0,119
7,2018-11-4 오후 09:00,“I’m not anti-gun. My great-grandmother taught...,156,475,5000,0,121,0,0,881
8,2018-11-4 오후 08:45,If Democratic nominee Gina Ortiz Jones for Con...,44,129,999,0,0,13,0,175
9,2018-11-4 오후 08:00,The father and daughter both claimed their rel...,186,108,100,347,0,168,0,0
